In [1]:
from diviner import file_utils as fu

import pandas as pd
import diviner as div
import dask

In [2]:
dask.__version__

'2.1.0'

In [3]:
fu.l1adatapath

PosixPath('/q/marks/feidata/DIV:opsL1A/data')

In [21]:
out = Path('/luna4/maye/calib_database')
out

PosixPath('/luna4/maye/calib_database')

In [22]:
from dask import delayed, compute

In [26]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:42977 Dashboard: http://127.0.0.1:40389/status,Cluster Workers: 4 Cores: 16 Memory: 101.37 GB


In [1]:
client.close()

NameError: name 'client' is not defined

In [27]:
def process_data(tstr):
    filenames = list(fu.l1adatapath.glob(tstr+'*'))
    delayed_results = []
    for fname in filenames: 
        divfile = fu.L1ADataFile(fname)
        df = delayed(divfile.read)()
        delayed_results.append(df[df.is_calib])
    return pd.concat(compute(*delayed_results))

In [29]:
def l1a_to_parquet(fname):
    l1a_parquet_dir = Path('/luna4/maye/l1a_parquet')
    divfile = fu.L1ADataFile(fname)
    savedir = l1a_parquet_dir / divfile.fn_handler.tstr[:4]
    savedir.mkdir(exist_ok=True)

    outfname = savedir / fname.with_suffix('.parquet').name
    if outfname.exists():
        return
    divfile.read().to_parquet(outfname)

In [30]:
dataset = fu.l1adatapath.glob('201*')
futures = client.map(l1a_to_parquet, dataset)

In [28]:
from dask import dataframe as dd

In [22]:
dd.read_csv(data.fname, names=data.header.columns, na_values='-9999', skiprows=8, skipinitialspace=True)

In [23]:
df = fu.parse_times(df)

NotImplementedError: Series getitem in only supported for other series objects with matching partition structure

In [32]:
nfiles_per_yr = []
for year in range(2009, 2020):
    nfiles_per_yr.append(len(list(fu.l1adatapath.glob(str(year)+'*'))))

In [34]:
nfiles_per_yr

[4297, 8585, 8679, 8719, 8697, 8652, 8760, 8784, 8760, 8737, 3557]

In [39]:
df = pd.DataFrame(nfiles_per_yr, index=range(2009, 2020), columns=['nfiles_per_yr'])
df

,nfiles_per_yr
2009,4297
2010,8585
2011,8679
2012,8719
2013,8697
2014,8652
2015,8760
2016,8784
2017,8760
2018,8737


In [40]:
nhours_per_year = []
for year in range(2009, 2020):
    nhours = 366*24 if year % 4 == 0 else 365*24
    nhours_per_year.append(nhours)

In [41]:
nhours_per_year

[8760, 8760, 8760, 8784, 8760, 8760, 8760, 8784, 8760, 8760, 8760]

In [42]:
df['nhours_per_yr'] = nhours_per_year

In [43]:
df

,nfiles_per_yr,nhours_per_yr
2009,4297,8760
2010,8585,8760
2011,8679,8760
2012,8719,8784
2013,8697,8760
2014,8652,8760
2015,8760,8760
2016,8784,8784
2017,8760,8760
2018,8737,8760


In [45]:
df['ratio'] = df.nfiles_per_yr / df.nhours_per_yr

In [46]:
df

,nfiles_per_yr,nhours_per_yr,ratio
2009,4297,8760,0.490525
2010,8585,8760,0.980023
2011,8679,8760,0.990753
2012,8719,8784,0.992600
2013,8697,8760,0.992808
2014,8652,8760,0.987671
2015,8760,8760,1.000000
2016,8784,8784,1.000000
2017,8760,8760,1.000000
2018,8737,8760,0.997374


In [48]:
import hvplot.pandas

In [50]:
df.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [51]:
df.to_csv('file_stats.csv')

In [52]:
pwd

'/u/paige/maye/src/diviner/notebooks/production'